# Exploring disease models

This notebook looks at a number of simple models to consider the spread of infectious diseases in a population, such as Covid-19 in the UK. These are not intended to provide highly accurate predictions of what will happen, but you can hopefully get an idea of the sorts of models researchers use and the questions we can ask with them. All of the models are based on the classic Susceptible-Infected-Recovered model, given by the following ordinary differential equations,

\begin{align*}
\frac{dS}{dt}&=-\beta SI \\
\frac{dI}{dt}&=\beta SI - \gamma I\\
\frac{dR}{dt}&=\gamma I.
\end{align*}

There are lots of assumptions built in this model, but a few key ones to note are:
* Immunity is permanent.
* There is no vaccination.
* There is no distinction between symptomatic and asymptomatic individuals.


In [7]:
#!jupyter nbextension enable --py widgetsnbextension --sys-prefix
#!jupyter serverextension enable voila --sys-prefix

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok
Enabling: voila
- Writing config: C:\ProgramData\Anaconda3\etc\jupyter
    - Validating...
      voila 0.3.0 ok


In [1]:
#@title
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import ipywidgets as widgets
from ipywidgets import interact_manual
#print('Libraries loaded')

## Control measures
We will start by looking at the basic dynamics of an epidemic with parameter values loosely based on Covid-19 in the UK: a population size $N$ of 65 million, an $R_0$ of 2.5 and an infectious period of 14 days. We will not add any more detail to the model than this. The red line shows what would happen with no interventions under the basic assumptions of the SIR model.

We will then ask what happens if we apply control measures that act to reduce this value of $R_0$ by some percentage. Initially we assume a 50\% reduction to $R_0$ (meaning $R_0=1.25$ during control), starting on day 100 and lasting for 6  months. This seems to work quite well as a control measure initially, but would mean half a year of significant restrictions to lower $R_0$ followed by a large outbreak later in the year. Try varying the values to see what might happen if we used different approaches.

In [2]:
#@title


controlstartw=widgets.FloatSlider(min=10, max=250, step=10, value=100, description='Control start day')
controllengthw=widgets.FloatSlider(min=0, max=360, step=10, value=180, description='Control length')
controleffectw=widgets.FloatSlider(min=0, max=100, step=5, value=50, description='Control effect')

def disease(x,t,BETA,GAMMA):
    sdot=-BETA*x[0]*x[1]
    idot=BETA*x[0]*x[1]-GAMMA*x[1]
    return sdot, idot 

def basic_covid():
  start_cont=int(controlstartw.value)
  end_cont=int(start_cont+controllengthw.value)

  N0=65000000        # Population size
  gamma=1/14         # Recovery rate
  R0=2.5
  scale=10**6 # Use to make plots easier to understand
  beta1=R0*gamma/N0  # Transmission
  ts=np.linspace(0,365,8000)
  x0=[N0-1,1]  #Initial condition assuming population of 65 million with 1 infected

  ta=np.linspace(0, start_cont, start_cont*10)
  tb=np.linspace(start_cont,end_cont,600)
  if end_cont<365:
    tc=np.linspace(end_cont,365,(365-end_cont)*10)
  else:
    tc=np.linspace(end_cont,end_cont+0.001,1)
  xxa=odeint(disease,x0,ta,args=(beta1,gamma,))
  xxb=odeint(disease,[xxa[-1,0],xxa[-1,1]],tb,args=((100-controleffectw.value)/100*beta1,gamma,))
  xxc=odeint(disease,[xxb[-1,0],xxb[-1,1]],tc,args=(beta1,gamma,))

  plt.rcParams['figure.figsize'] = [12, 4]#
  plt.rcParams.update({'font.size': 16})
  
  fig1=plt.figure()
  xx1=odeint(disease,x0,ts,args=(beta1,gamma,))
  plt.plot(ts,xx1[:,1]/scale,'r',label='No control')
  plt.plot(ta,(xxa[:,1]/scale),'b',label='Control')
  plt.plot(tb,(xxb[:,1]/scale),'b')
  plt.plot(tc,(xxc[:,1]/scale),'b')
  plt.xlabel('Days')
  plt.ylabel('No. Infected individuals (millions)')
  plt.axis([0,365,0,16])

display(controlstartw,controllengthw)
display(controleffectw)
my_interact_manual = interact_manual.options(manual_name="Run model")
my_interact_manual(basic_covid);

FloatSlider(value=100.0, description='Control start day', max=250.0, min=10.0, step=10.0)

FloatSlider(value=180.0, description='Control length', max=360.0, step=10.0)

FloatSlider(value=50.0, description='Control effect', step=5.0)

interactive(children=(Button(description='Run model', style=ButtonStyle()), Output()), _dom_classes=('widget-i…

## Control in high-risk and low-risk groups

While Covid-19 can cause serious illness in anyone, data shows that it is those who have underlying health conditions who tend to experience the most severe outcomes. One suggested approach for controlling spread is to target control measures at the most vulnerable in a bid to protect them, while allowing the rest of the population to continue with fewer restrictions.

In this mdoel we assume 20\% of the population are deemed 'high-risk' and the rest 'low-risk'. Control measures can be applied to just one of these two risk groups, to both or to neither. You can use the slider to adjust the degree of control. The output will show both the total number of infections under each type of control measure as well as the number of infections just in the high-risk group. Does the strategy of targeting control at high-risk groups work?

In [1]:
#@title
# A low-risk/high-risk group model

controlw=widgets.FloatSlider(min=0, max=50, step=5, value=25, description='Control')

def disease_risk(x,t,betall,betahh,betahl,gamma):
    sldot=-betall*x[0]*x[1]-betahl*x[0]*x[3]
    ildot=betall*x[0]*x[1]+betahl*x[0]*x[3]-gamma*x[1]
    shdot=-betahh*x[2]*x[3]-betahl*x[2]*x[1]
    ihdot=betahh*x[2]*x[3]+betahl*x[2]*x[1]-gamma*x[3]
    return sldot, ildot, shdot, ihdot

def riskmodel():
  N0=65000000        # Population size
  gamma=1/14         # Recovery rate
  R0=2.5
  scale=10**6 # Use to make plots easier to understand
  beta1=R0*gamma/N0  # Transmission
  ts=np.linspace(0,365,8000)
  x0=[N0*0.8-1,1,N0*0.2,0]  #Initial condition
  control=(100-controlw.value)/100

  xx1=odeint(disease_risk,x0,ts,args=(beta1,beta1,beta1,gamma,))
  xx2=odeint(disease_risk,x0,ts,args=(control*beta1,beta1,control*beta1,gamma,))
  xx3=odeint(disease_risk,x0,ts,args=(beta1,control*beta1,control*beta1,gamma,))
  xx4=odeint(disease_risk,x0,ts,args=(control*beta1,control*beta1,control*beta1,gamma,))

  fig1=plt.figure()
  plt.plot(ts,(xx1[:,1]+xx1[:,3])/scale,'r',label='No control')
  plt.plot(ts,(xx3[:,1]+xx3[:,3])/scale,'y',label='Control in high-risk')
  plt.plot(ts,(xx2[:,1]+xx2[:,3])/scale,'b',label='Control in low-risk')
  plt.plot(ts,(xx4[:,1]+xx4[:,3])/scale,'k',label='Control in all')

  plt.xlabel('Days')
  plt.ylabel('No. Infected individuals (millions)')
  plt.legend();

  fig2=plt.figure()
  plt.plot(ts,(xx1[:,3]/scale),'r',label='No control')
  plt.plot(ts,(xx3[:,3]/scale),'y',label='Control in high-risk')
  plt.plot(ts,(xx2[:,3]/scale),'b',label='Control in low-risk')
  plt.plot(ts,(xx4[:,3]/scale),'k',label='Control in all')

  plt.xlabel('Days')
  plt.ylabel('No. high-risk Infected individuals (millions)')
  plt.legend();
  return()

display(controlw)
my_interact_manual2 = interact_manual.options(manual_name="Run model")
my_interact_manual2(riskmodel);

NameError: name 'widgets' is not defined

## Deterministic vs stochastic models

The SIR ODE model assumes the epidemic is entirely deterministic, meaning if we run the model twice with the same values the exact same outcome occurs. Often we'd like to use a stochastic model, where 'events' happen according to certain probabilities, acounting for the inherent randomness in biological processes. Such dynamics often `look' much more realistic, but are harder to analyse in depth. 

A question we might ask, therefore, is how well an ODE model does at capturing the output of a stochastic model. The code below explores this. It assumes a population of 1000 people and an infectious period of 14 days. You should find that when $R_0$ is high, the ODE model does a very good job as an 'average' of the individual-based runs. When $R_0$ is lower there is much more variability, but the ODE model still does ok as a guide.

In [5]:
#@title
# Import the necessary libraries

R0w=widgets.FloatSlider(min=0, max=10, step=0.5, value=2.5, description='R_0')   

def run_model1():
  N=1000           
  GAMMA=1/14
  R0=R0w.value
  BETA=R0*GAMMA/N
  REPS=20
  I0=5


  # INDIVIDUAL-BASED MODEL

  # Function to count noumbers in each compartment
  def count_type(type):
      current_type=0;
      for i in range(0,N):
          for j in range(0,N):
              if grid[i,j]==type:
                  current_type=current_type+1
      return current_type

  # Function to check the current scale
  def findscale():
      S=susceptibles[-1]
      I=infecteds[-1]
      #Set relative parameter values
      scale=GAMMA*I+BETA*S*I  
      return scale

  for reps in range(0,REPS):
      # Set initial conditions
      #grid=np.zeros((N,N))
      tsteps=[0]
      infecteds=[I0]
      susceptibles=[N-I0]
      current_t=0

      plt.rcParams['figure.figsize'] = [12, 4]#
      plt.rcParams.update({'font.size': 16})

      # Main run
      while current_t<150:
          # Find tau-leap
          scale=findscale()
          dt = -np.log(np.random.rand()) / scale
          current_t=tsteps[-1]
          tsteps.append(dt+current_t)

          #Find event
          if np.random.rand()<GAMMA*infecteds[-1]/scale: #Event is recovery  
              infecteds.append(infecteds[-1]-1)
              susceptibles.append(susceptibles[-1])
          else: #Event is transmission
              infecteds.append(infecteds[-1]+1)
              susceptibles.append(susceptibles[-1]-1)

          if infecteds[-1]==0:
              break

      # Plot latest run 
      if reps==0:
          plt.plot(tsteps,infecteds,':',color='lightgray',label='Stochastic model')
      else:
          plt.plot(tsteps,infecteds,':',color='lightgray')

  ts=np.linspace(0,150,2000)
  xx1=odeint(disease,[N-I0,I0],ts,(BETA,GAMMA,)) 
  plt.plot(ts,(xx1[:,1]),'k',label='ODE model')    
  plt.legend()
  plt.xlabel('Time')
  plt.ylabel ('No. Infected')
  plt.xlim(0,150)
  return()

display(R0w)
my_interact_manual3 = interact_manual.options(manual_name="Run model")
my_interact_manual3(run_model1)      


FloatSlider(value=2.5, description='R_0', max=10.0, step=0.5)

interactive(children=(Button(description='Run model', style=ButtonStyle()), Output()), _dom_classes=('widget-i…

<function __main__.run_model1()>

## Local interactions

This section summarises work conducted by an undergraduate student at the School of Mathematics and Statistics during their summer project with me, and published in *Bulletin of Mathematical Biology*.

Standard models we have looked at so far assume the infection spreads 'globally' - that is, as a suceptible individual you are equally likely to catch the infection off any infected individual. In reality, you are much more likely to catch the disease off somebody physically close to you. A simple way of modelling this is to assume the population lives on a square lattice. An individual's contacts could then be *either* global across the lattice *or* local with its 4 near-neighbours, or a mix of the two. 

This code introduces a parameter $L$ to control the degree of local interactions. $L=0$ means all contacts are global, $L=1$ means all contacts are local, and $0<L<1$ means a weigthed mix of the two. 

We present this model in 3 different ways:


1.   The standard ODE model that does not include any structure.
2.   An approximate ODE model using a 'pair approximation' to include some structure.
3. 10 runs of a stochastic model that is fully spatially structured.

*This can take a few seconds to run.*

In [17]:
#@title
# Import the necessary libraries
#from IPython.display import HTML
plt.rcParams.update({'font.size': 16})

Lw=widgets.FloatSlider(min=0, max=1, step=0.05, value=0, description='L')  

gridsaves=np.zeros([36,20,20])
num_frames=0

def runspatial():
  N=20                     # Grid size (suggested min. of 20 and max. of 50 - the larger, the longer it takes)
  tot=N*N
  GAMMA=1/14               # Recovery rate
  R0=2.5                   # R_0
  BETA=R0*GAMMA/tot        # Transmission
  I0=5                     # Starting no. of infecteds
  REPS=10                  # No. of replicates of IBM to do
  L=Lw.value

  # PAIR APPROXIMATION
  def pairs(x,t,BETAP):
      qis=x[3]/x[0]
      sdot=-BETAP*x[0]*(L*qis+(1-L)*x[1])
      idot=BETAP*x[0]*(L*qis+(1-L)*x[1])-GAMMA*x[1]
      ssdot=-2*BETAP*(L*3/4*qis+(1-L)*x[1])*x[2]
      sidot=-BETAP*(L*(1/4+3/4*qis)+(1-L)*x[1])*x[3]-GAMMA*x[3]+BETAP*(L*3/4*qis+(1-L)*x[1])*x[2]
      srdot=-BETAP*(L*3/4*qis+(1-L)*x[1])*x[4]+GAMMA*x[3]
      iidot=-2*GAMMA*x[5]+2*BETAP*(L*(1/4+3/4*qis)+(1-L)*x[1])*x[3]
      irdot=-GAMMA*x[6]+BETAP*(L*3/4*qis+(1-L)*x[1])*x[4]+GAMMA*x[5]
      return [sdot,idot,ssdot,sidot,srdot,iidot,irdot]
      
  # MEAN-FIELD APPROXIMATION
  def disease(x,t,BETAP):
      sdot=-BETAP*x[0]*x[1]
      idot=BETAP*x[0]*x[1]-GAMMA*x[1]
      return sdot, idot

  # Function to count noumbers in each compartment
  def count_type(sirtype):
      current_type=0;
      for i in range(0,N):
          for j in range(0,N):
              if grid[i,j]==sirtype:
                  current_type=current_type+1
      return current_type

  # Function to find nearest-neighbours
  def local_inf(site_x,site_y):
      localinfs=0
      if site_x==0:
          downx=N-1
      else:
          downx=site_x-1
      if site_y==0:
          downy=N-1
      else:
          downy=site_y-1
      if site_x==N-1:
          upx=0
      else:
          upx=site_x+1
      if site_y==N-1:
          upy=0
      else:
          upy=site_y+1

      if grid[downx,site_y]==1:
          localinfs=localinfs+1
      if grid[upx,site_y]==1:
          localinfs=localinfs+1
      if grid[site_x,downy]==1:
          localinfs=localinfs+1
      if grid[site_x,upy]==1:
          localinfs=localinfs+1
      return localinfs

  # Function to check the current scale
  def findscale():
      S=count_type(0)
      I=count_type(1)
      localinf=0
      for i in range(0,N):
          for j in range(0,N):
              if grid[i,j]==0:
                  localinf=local_inf(i,j)+localinf
      if S>0:
          QIS=localinf/S*tot/4
      else:
          QIS=0
      #Set relative parameter values
      scale=GAMMA*I+BETA*S*(L*QIS+(1-L)*I)  
      return scale

  #saves up to 500 plots for animation
  global gridsaves
  sus,inf,rec,tplot = [],[],[],[]
  global num_frames

  gridsaves=np.zeros([90,20,20])
  num_frames=0


  ibmmax=np.zeros(REPS)
  ibmt=np.zeros(REPS)

  # Multiple runs of model
  for reps in range(0,REPS):
      if reps==0:
          print('Running simulation 1', end=" ")
      else:
        print(', %i' % int(reps+1),end=" ")
      plots=-1
      # Set initial conditions
      init_inf=I0
      grid=np.zeros((N,N))
      for i in range(0,init_inf):
          grid[np.random.randint(0,N-1),np.random.randint(0,N-1)]=1
      tsteps=[0]
      infecteds=[count_type(1)]
      current_t=0

      # Main run
      while current_t<180:

          # Find tau-leap
          scale=findscale()
          dt = -np.log(np.random.rand()) / scale
          current_t=tsteps[-1]

          #saves a grid every 2 days
          for k in range(90):
              if reps==0 and current_t<k*2 and current_t+dt>k*2:
                  gridsaves[num_frames][:][:]=grid[:][:]
                  num_frames +=1
                  sus.append(count_type(0)/(N*N))
                  inf.append(count_type(1)/(N*N))
                  rec.append(count_type(2)/(N*N))
                  tplot.append(dt+current_t)

          # Create randomised list of sites to check
          findx=[i for i in range(N)]
          findy=[i for i in range(N)]
          np.random.shuffle(findx)
          np.random.shuffle(findy) 
          flagged=0   # Used to break out of 2nd loop

          #Find event
          if np.random.rand()<GAMMA*infecteds[-1]/scale: #Event is recovery   
              for tryx in findx:
                  if flagged==1:
                      break
                  for tryy in findy:
                      if grid[tryx,tryy]==1:
                          grid[tryx,tryy]=2
                          flagged=1
                          break
          else: #Event is transmission
              if np.random.rand()>L: #Transmission is global
                  for tryx in findx:
                      if flagged==1:
                          break
                      for tryy in findy:
                          if grid[tryx,tryy]==0:                       
                              grid[tryx,tryy]=1
                              flagged=1
                              break
              else: # Transmission is local
                  for tryx in findx:
                      if flagged==1:
                          break
                      for tryy in findy:
                          if grid[tryx,tryy]==0: 
                              if local_inf(tryx,tryy)>0:
                                  grid[tryx,tryy]=1
                                  flagged=1
                                  break

          # Update time and infection lists
          tsteps.append(dt+current_t)
          infecteds.append(count_type(1))
          if infecteds[-1]==0:
              break

      # Plot latest run
      infected_prop=[inf/tot for inf in infecteds]
      ibmmax[reps]=max(infected_prop)
      for j in range(0,len(tsteps)):
          if infected_prop[j]==ibmmax[reps]:
              ibmt[reps]=tsteps[j]
              break
      if reps==0:
        plt.plot(tsteps,infected_prop,'b:',alpha=0.3,label='Stochastic model') 
      else:
        plt.plot(tsteps,infected_prop,'b:',alpha=0.3)   



  ETA=R0*GAMMA # In ODE models total pop=1 so beta needs changing
  ts=np.linspace(0,180,2000)
  xx1=odeint(disease,[1-I0/tot,I0/tot],ts,args=(ETA,)) 
  plt.plot(ts,(xx1[:,1]),'k',label='Mean-field')



  ts=np.linspace(0,180,2000)
  xxp=odeint(pairs,[1-I0/tot,I0/tot,(1-I0/tot)**2,(1-I0/tot)*(I0/tot),0,(I0/tot)**2,0],ts,args=(ETA,)) 
  plt.plot(ts,(xxp[:,1]),'r',label='Spatial approx.')
  plt.xlabel('Days')
  plt.ylabel('Proportion Infected')
  plt.legend()
  plt.ylim(0,0.4)
  plt.xlim(0,180)
  #plt.savefig('l09.png')
  plt.show()


  print('By day 180,', +int(100-100*xxp[-1,0]), 'per-cent of the population have been infected.')
  return()

display(Lw)
my_interact_manual4 = interact_manual.options(manual_name="Run model")
my_interact_manual4(runspatial);

FloatSlider(value=0.0, description='L', max=1.0, step=0.05)

interactive(children=(Button(description='Run model', style=ButtonStyle()), Output()), _dom_classes=('widget-i…

In [15]:
#@title
#animations for the lattice for the IBM

import matplotlib.colors as clt
from matplotlib import animation, rc
import matplotlib.patches as patches
from IPython.display import HTML

def make_anim():
    print('Building animation - please wait')
    def animate(i):
        N=20
        ax.clear()
        # make color map
        my_cmap = clt.ListedColormap(['b', 'r', 'y'])
        # set the 'bad' values (nan) to be white and transparent
        my_cmap.set_bad(color='w', alpha=0)
        for x in range(N + 1):
            ax.axhline(x, lw=1, color='k', zorder=5)
            ax.axvline(x, lw=1, color='k', zorder=5)    
        # draw the boxes
        ax.imshow(gridsaves[i], interpolation='none', cmap=my_cmap, extent=[0, N, 0, N], zorder=0)
        #legend
        names = ['Susceptible','Infected','Recovered']
        pat = [ patches.Patch(color=['b', 'r', 'y'][j], label="{l}".format(l=names[j]) ) for j in range(len(names)) ]
        ax.legend(handles=pat, loc=6, bbox_to_anchor=(1.05, 0.5), borderaxespad=0. )
        #title and axis
        ax.set_title('Simulated infections through time and space') 
        ax.axis('off')


    fig, ax = plt.subplots(figsize=(10,5))
    plt.close(fig) #needs to be closed or it just prints a stationairy frame
    #animator - change repeat to false to make video stop at the end
    anim = animation.FuncAnimation(ax.figure, animate,interval=500,frames=int(num_frames),repeat=False)
    #HTML function for Jupyter
    display(HTML(anim.to_html5_video()))
    #anim.save('local0.mp4')

my_interact_manual5 = interact_manual.options(manual_name="Animate model")
my_interact_manual5(make_anim);

interactive(children=(Button(description='Animate model', style=ButtonStyle()), Output()), _dom_classes=('widg…